In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ✅ API Key
api_key = "-"

# ✅ Session with Retry
session = requests.Session()
retries = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount("http://", HTTPAdapter(max_retries=retries))

# ✅ SpotInfo 수집
spot_url = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/SpotInfo/1/1000/"
res = session.get(spot_url, timeout=10)
root = ET.fromstring(res.content)

spot_list = []
for row in root.findall("row"):
    spot_list.append({
        "spot_num": row.find("spot_num").text,
        "spot_name": row.find("spot_nm").text,
        "x": float(row.find("grs80tm_x").text),
        "y": float(row.find("grs80tm_y").text),
    })

spot_df = pd.DataFrame(spot_list)

GRS80TM → WGS84 (위도, 경도) 로 좌표 변환
(서울시 API 등에서 받은 교통량 좌표는 GRS80TM 기준이므로 변환 필요)

위경도 → 행정구/동 주소 (역지오코딩)
(Kakao API 또는 Naver Reverse Geocoding API 사용)

In [ ]:
from pyproj import Transformer

# 좌표 변환기 정의: GRS80 (EPSG:2097) → WGS84 (EPSG:4326)
transformer = Transformer.from_crs("epsg:2097", "epsg:4326", always_xy=True)

# 위경도 추가
spot_df["lon"], spot_df["lat"] = transformer.transform(spot_df["x"].values, spot_df["y"].values)

import requests
import time

# ✅ Kakao REST API 키 입력 (직접 발급 필요)
KAKAO_API_KEY = "a8db9510a269b4a52a329d5db7562692"

def reverse_geocode_kakao(lat, lon):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"x": lon, "y": lat}
    try:
        res = requests.get(url, headers=headers, params=params)
        result = res.json()
        if "documents" in result and len(result["documents"]) > 0:
            region = result["documents"][0]
            return region["region_2depth_name"], region["region_3depth_name"]
        else:
            return None, None
    except Exception as e:
        print(f"❗ 오류: {e}")
        return None, None

# ✅ 반복 적용
gu_list, dong_list = [], []
for idx, row in spot_df.iterrows():
    gu, dong = reverse_geocode_kakao(row["lat"], row["lon"])
    gu_list.append(gu)
    dong_list.append(dong)
    time.sleep(0.2)  # ✅ 너무 빠른 호출 방지

spot_df["구"] = gu_list
spot_df["동"] = dong_list

print(spot_df.head())




In [ ]:
# 기준 설정
lat_split = 37.55
lon_split = 127.00

def get_region_single(lat, lon):
    if lat >= lat_split and lon < lon_split:
        return "북서부"
    elif lat >= lat_split and lon >= lon_split:
        return "북동부"
    elif lat < lat_split and lon < lon_split:
        return "남서부"
    else:
        return "남동부"

# 적용
spot_df["지역"] = spot_df.apply(lambda row: get_region_single(row["lat"], row["lon"]), axis=1)

# 저장
spot_df.to_csv("spot_with_address.csv", index=False, encoding="utf-8-sig")

spot_df